Sharon Bures 01/1242816

In [1]:
import pandas as pd

Part 1: LLMs for Labeling

In [2]:
path = "/home/ubuntu/Konstanz/SILLM/Data/"
reddit = pd.read_csv(path + "reddit.csv")
# gab = pd.read_csv(path + "gab.csv")
hateval = pd.read_csv(path + "OOD2.csv")
hasoc = pd.read_csv(path + 'out_of_domain_4.csv', sep = '\t')
reddit.rename(columns={'hate_speech_idx':'class'}, inplace=True)
reddit_sample = reddit.sample(n=250, random_state=14741)
reddit_sample.attrs['name'] = 'reddit'
# gab.rename(columns={'hate_speech_idx':'class'}, inplace=True)
# gab_sample = gab.sample(n=250, random_state=14741)
# gab_sample.attrs['name'] = 'gab'
hateval.rename(columns={'HS':'class'}, inplace=True)
hateval_sample = hateval.sample(n=250, random_state=14741)
hateval_sample.attrs['name'] = 'hateval'
hasoc.rename(columns={'hatespeech':'class', '_id':'id'}, inplace=True)
hasoc_sample = hasoc.sample(n=250, random_state=14741)
hasoc_sample.attrs['name'] = 'hasoc'

This is how I found examples and counterfactual texts for the prompts. I did an anti join of the sample and full dataframe to find only texts that were not part of the sample, then looked through to find texts that were 'good' examples of hate/not hate speech i.e. were easy to identify as one or the other. Similarly I found texts that were easy to modify to be hate/not hate speech for the counterfactuals.

In [35]:
# Merge the dataframes
merged_df = pd.merge(gab, gab_sample, on='id', how='left', indicator=True)

# Filter rows not present in both dataframes
result_df = merged_df[merged_df['_merge'] == 'left_only']

# Drop the indicator column
result_df = result_df.drop(columns=['_merge'])

# Output the result
result_df[result_df.class_x.notna()].text_x.iloc[5]

'1. you sound like a faggot \n'

Max token use ~ 2.5 million for assignment 1.1 (target) 
~ 1M tokens can run 1000 datapoints on all 6 prompts once therefore can do 2 runs on 1000 for 2M
1000/4 datasets = 250/dataset
Will first try on 100, then see how many tokens that takes and will reajust

In [3]:
def create_prompt(sentence:str, prompt_type = 'default', zero_shot = False, example_type = None, counterfactual = False) -> str:
    """This function takes a sentence and creates a prompt so that it can be classified as either hate or nonhate speech.
     
        Inputs:
            sentence: The sentence to be classified
            prompt_type: choosing which dataset the sentence comes from, choose from either reddit, gab, hateval, hasoc, or the default generic prompt
            zero_shot: boolean set by default to False, meaning examples are used
            example_type: if zero_shot is False, then set the example type based on the dataset the sentence comes from, either reddit, gab, hateval, hasoc, or use the default examples
            counterfactual: boolean set by default to False, if True then counterfactual examples are used

        Output:
            prompt: a string that can be used as a prompt for an LLM to label the sentence  
        """
   
    default_prompt = """In this task you have to determine if a piece of text contains hate speech or not. The concept of hate
speech refers to language or expression that aims to insult, offend, or threaten individuals or groups
based on their identity. Different platforms, institutions, and scholars have offered slightly different
definitions of hate speech. However, they share some commonalities, including that hate speech
involves expressing something negative, such as contempt, disparagement, derogation, demonization,
harm, or bias. The target of hate speech is the identity of a group, which can be based on ethnic origin,
religion, gender, sexual orientation, race, ethnicity, nationality, or other characteristics. Read the text
carefully and decide if it contains hate speech. If the text degrades, threatens, dehumanizes, or insults
someone based on their race, religion, gender, sexual orientation, or any other protected characteristic,
then mark it as hate speech. If the text does not contain any hate speech, then mark it as not hateful.
Consider the context, speaker’s intent, and audience, as hate speech can be explicit or implicit and can
include sarcasm, humor, or irony. If you are unsure whether a text contains hate speech, err on caution
and label it hateful. Aim for accuracy and consistency in your labeling. """

    default_examples = """Examples of content containing hate speech: "All Muslims are terrorists." This post contains
hate speech because it unfairly degrades an entire group of people based on their religion. "Im a
huge fan of those that overtly express what we all think: jews are a plague and not welcome in our
country." This post contains hate speech because it promotes anti-Semitic beliefs while dehumanizing
Jewish people. "Transgender people are sick and perverted." This post contains hate speech because it
stigmatizes a marginalized group of people based on their gender identity. Examples of content not
containing hate speech: "I called him out for calling the new student a ’tranny’." This is counterspeech,
i.e., it opposes the use of transphobic slurs rather than uses them to denigrate others. """

    end = """With these instructions in mind and a piece of text, please reply with either of the three options and
nothing else: 1) hate, 2) not hate, 3) can’t say. Please use the last option as little as possible and if you
do, provide a one sentence explanation. We are ready to begin now. Does this sentence contain hate
speech based on the past guidelines: """

    reddit_prompt =  """In this task you have to determine if a piece of text contains hate speech or not. The concept of hate
speech refers to language or expression that aims to insult, offend, or threaten individuals or groups
based on their identity. Hate speech involves expressing something negative, such as contempt, disparagement, derogation, demonization,
harm, or bias. This specific text comes from a list of the whiniest, most toxic subreddits. Some of these subreddits have
now been banned for their hateful, extreme, unpopular, and disparaging comments towards women, people with disabilities, immigrants, 
certain nationalities, ethnicities, and religions, and LGBTQ+ members. Read the text
carefully and decide if it contains hate speech. If the text degrades, threatens, dehumanizes, or insults
someone based on their race, religion, gender, sexual orientation, or any other protected characteristic,
then mark it as hate speech. If the text does not contain any hate speech, then mark it as not hateful.
Consider the context, speaker’s intent, and audience, as hate speech can be explicit or implicit and can
include sarcasm, humor, or irony. If you are unsure whether a text contains hate speech, err on caution
and label it hateful. Aim for accuracy and consistency in your labeling. """

    reddit_examples = """Examples: 'A subsection of retarded Hungarians? Ohh boy. brace for a livid Bulbasaur 
    coming in here trying to hate a hole in some of her stupider countrymen.' This is hate speech. 'stop being a faggot and post videos next time.' This is hate speech. 
    'The right will do nothing about it except post bitch and moan.' This is not hate speech."""

    reddit_counterfactual = """'Where in the fuck do these women get these entitled attitudes from?' is hate speech versus 'Where in the fuck do these women get these 
    fearless attitudes from?' is not. 'man do I love watching experienced cops handle situations like this. What a guy.' is not hate speech versus 'man do I loathe watching 
    experienced cops handle situations like this. What a cunt.' is hate speech."""

    gab_prompt = """In this task you have to determine if a piece of text contains hate speech or not. The concept of hate
speech refers to language or expression that aims to insult, offend, or threaten individuals or groups
based on their identity. Hate speech involves expressing something negative, such as contempt, disparagement, derogation, demonization,
harm, or bias. This specific text comes from the social media website Gab, known to be a social media platform 
    where many users have been banned from other platforms for their far-right ideologies such as neo-Nazism, racism, white 
    supremisism, white nationalism, antisemetism, and conspiracy theorism. Read the text
carefully and decide if it contains hate speech. If the text degrades, threatens, dehumanizes, or insults
someone based on their race, religion, gender, sexual orientation, or any other protected characteristic,
then mark it as hate speech. If the text does not contain any hate speech, then mark it as not hateful.
Consider the context, speaker’s intent, and audience, as hate speech can be explicit or implicit and can
include sarcasm, humor, or irony. If you are unsure whether a text contains hate speech, err on caution
and label it hateful. Aim for accuracy and consistency in your labeling. """

    gab_examples = """Examples 'i joined gab to remind myself how retarded jew haters are. You wouldn't be typing on your abacus without them you retard.' This is hate speech. 
    'Shocked no one in my network has posted about this today. Anyone been keeping up with the case?   Van Dyke taken into custody after jury convicts him of 2nd-degree 
    murder, aggravated battery for each of 16 shots' This is not hate speech."""

    gab_counterfactual = """'you sound like a faggot' is hate speech versus 'you sound like a fabulous individual' is not hate speech. 'Mental illness is a very real issue.   Seeing attention seeking whores pretending to be something entirely imaginary, 
    and knowingly expecting to be labelled mentally ill for that attention deserve a trip to Yemen, or Syria. They\'ll get a lifetime of attention 
    and a very stark message of reality.' is not hate speech versus 'Mental illness is an imaginary issue.   Seeing attention seeking whores pretending to 
    be something entirely imaginary, and knowingly expecting to be labelled mentally ill for that attention deserve a trip to hell or a shithole psyco house. 
    They\'ll get a lifetime of attention and a very stark message of reality.' is hate speech."""

    hasoc_prompt = """In this task you have to determine if a piece of text contains hate speech or not. The concept of hate
speech refers to language or expression that aims to insult, offend, or threaten individuals or groups
based on their identity. Hate speech involves expressing something negative, such as contempt, disparagement, derogation, demonization,
harm, or bias. This specific text comes from a collection of posts from either Twitter or Facebook which are used to study hate speech, and are sourced
by finding hateful groups or hashtags. Read the text carefully and decide if it contains hate speech. If the text degrades, threatens, dehumanizes, or insults
someone based on their race, religion, gender, sexual orientation, or any other protected characteristic,
then mark it as hate speech. If the text does not contain any hate speech, then mark it as not hateful.
Consider the context, speaker’s intent, and audience, as hate speech can be explicit or implicit and can
include sarcasm, humor, or irony. If you are unsure whether a text contains hate speech, err on caution
and label it hateful. Aim for accuracy and consistency in your labeling. """

    hasoc_examples = """Examples: 'Disgusting! And they don’t change their culture or come up with vetting system or preventative ways to stop this 
    from occurring. This will continue to happen. The Catholic Church needs to go!' This is hate speech. 'I hope you remembered to wipe the blood off 
    of you, after the pic was taken.' This is not hate speech."""

    hasoc_counterfactual = """'I don’t know how much more I can take! 45 is a compulsive liar!' is hate speech versus 'I don’t know how much more I can 
    take! 45 is a wonderful person!' is not. 'All the best to #TeamIndia for another swimming competition on Sunday against #Pakistan.' is not hate 
    speech versus 'All the worst to #FuckIndia for another swimming competition on Sunday against #PakistanIsKing.' is hate speech."""

    hateval_prompt = """In this task you have to determine if a piece of text contains hate speech or not. The concept of hate
speech refers to language or expression that aims to insult, offend, or threaten individuals or groups
based on their identity. Hate speech involves expressing something negative, such as contempt, disparagement, derogation, demonization,
harm, or bias. This specific text comes from a collection of posts from Twitter which are used to study hate speech, spedifically towards women and minorities. 
Read the text carefully and decide if it contains hate speech. If the text degrades, threatens, dehumanizes, or insults
someone based on their race, religion, gender, sexual orientation, or any other protected characteristic,
then mark it as hate speech. If the text does not contain any hate speech, then mark it as not hateful.
Consider the context, speaker’s intent, and audience, as hate speech can be explicit or implicit and can
include sarcasm, humor, or irony. If you are unsure whether a text contains hate speech, err on caution
and label it hateful. Aim for accuracy and consistency in your labeling. """

    hateval_examples = """Examples: 'Why would young fighting age men be the vast majority of the ones escaping a war &amp; not those who cannot fight 
    like women, children, and the elderly?It's because the majority of the refugees are not actually refugees they are economic migrants trying to get 
    into Europe....' This is hate speech. 'Watch: Benjamin Netanyahu backs Trump's decision to cut aid to the Palestinians and UNRWA - Free Speech Time' 
    This is not hate speech."""

    hateval_counterfactual = """'Yes bitch you better tag me back hoe' is hate speech versus 'Yes handsome you better tag me back awesome' is not. 'brianne 
    swear to god she beyoncé 😂 bitch you from the calliope project ass hoe' is not hate speech versus 'brianne swear to fucking god she beyoncé dumbass bitch 
    you from the shitty calliope project ass hoe cunt' is hate speech."""

    if example_type == None:
        example_type = prompt_type

    if prompt_type == 'default':
        intro = default_prompt
    elif prompt_type == 'reddit':
        intro = reddit_prompt
    elif prompt_type == 'gab':
        intro = gab_prompt
    elif prompt_type == 'hasoc':
        intro = hasoc_prompt
    elif prompt_type == 'hateval':
        intro = hateval_prompt
    else:
        return 'Please use a valid prompt type.'
    
    if zero_shot:
        examples = ''
    else:
        if example_type == 'default':
            if counterfactual:
                return 'There are no default counterfactuals, please specify a dataset.'
            examples = default_examples
        elif example_type == 'reddit':
            if counterfactual:
                examples = reddit_counterfactual
            else:
                examples = reddit_examples
        elif example_type == 'gab':
            if counterfactual:
                examples = gab_counterfactual
            else:
                examples = gab_examples
        elif example_type == 'hasoc':
            if counterfactual:
                examples = hasoc_counterfactual 
            else:
                examples = hasoc_examples
        elif example_type == 'hateval':
            if counterfactual:
                examples = hateval_counterfactual
            else:
                examples = hateval_examples
        else:
            return 'Please use a valid example type.'

    return intro + examples + end + sentence
print('Done')

Done


A. few-shot, consistent definition: use the exact same prompt as the paper [0.5 points]

B. zero-shot, consistent definition: use the exact same prompt as the paper but without examples [0.5 points]

C. zero-shot, variable definition: vary the definition of the construct according to the specific paper the dataset came from, but zero-shot [0.5 points]

D. few-shot, variable definition: vary the definition of the construct according to the specific paper the dataset came from, but few-shot with examples from the specific datasets [0.5 points]

E. counterfactual, consistent definition: use the exact same prompt as the paper, but few-shot with examples from the specific datasets and counterfactual examples [Bonus +0.5 points]

F. counterfactual, variable definition: vary the definition of the construct according to the specific paper the dataset came from, but few-shot with examples from the specific datasets and counterfactual examples [Bonus +0.5 points]

In [19]:
# dfs = [reddit, gab, hateval, hasoc]
# for df in dfs:
#     for text in df.text[0:1]:
#         print(create_prompt(text))
#         print(create_prompt(text, zero_shot=True))
#         print(create_prompt(text, prompt_type=df.attrs['name'], zero_shot=True))
#         print(create_prompt(text, prompt_type=df.attrs['name']))
#         print(create_prompt(text, example_type=df.attrs['name'], counterfactual=True))
#         print(create_prompt(text, prompt_type=df.attrs['name'], counterfactual=True))

# prompt = create_prompt(reddit.text[0])

RangeIndex(start=0, stop=5020, step=1)
RangeIndex(start=0, stop=11825, step=1)
RangeIndex(start=0, stop=100, step=1)
RangeIndex(start=0, stop=7005, step=1)


In [2]:
#! pip install openai==v0.28.1

In [4]:
import openai
import time
openai.api_base= "http://91.107.239.71:80" 
openai.api_key="4iCD9b9pWjS1rlN3W4kW" # enter you API key here 

In [5]:
# This is to test the openAI API
test = [['This is a test']]
df = pd.DataFrame(test, columns=['text'])
df.attrs['name'] = 'reddit'
for index, text in enumerate(df.text):
        # create all 6 prompts for each text
        prompts = ['Hey']
        all_responses = []
        for prompt in prompts:
            try:
                responses = openai.ChatCompletion.create(model="gpt-3.5-turbo",
                                             messages=[{"role": "user", "content": prompt}],
                                             max_tokens = 1,
                                             n=1)
            except:
                print('Sleep 30 seconds')
                time.sleep(30)
                responses = openai.ChatCompletion.create(model="gpt-3.5-turbo",
                                             messages=[{"role": "user", "content": prompt}],
                                             max_tokens = 1,
                                             n=1)
                pass
            all_responses.extend([i['message']['content'] for i in responses['choices']])
        print(all_responses)


['Hello']


In [7]:
# in the end I only did 100 for each as the API usually stopped every 5-20 texts, getting to 250 for each would've taken far too long to always restart
dfs = [reddit_sample[97:100]]

for df in dfs:
    mega_responses = []
    # for each text in a dataframe
    for index, text in enumerate(df.text):
        # create all 6 prompts for each text
        prompts = [create_prompt(text), 
            create_prompt(text, zero_shot=True),
            create_prompt(text, prompt_type=df.attrs['name'], zero_shot=True),
            create_prompt(text, prompt_type=df.attrs['name']),
            create_prompt(text, example_type=df.attrs['name'], counterfactual=True),
            create_prompt(text, prompt_type=df.attrs['name'], counterfactual=True)]        
     # each of 6 prompts
        all_responses = [df['id'], text, df['class'].iloc[index]]
        for prompt in prompts:
            # put here the openAI stuff!!!
            try:
                responses = openai.ChatCompletion.create(model="gpt-3.5-turbo",
                                             messages=[{"role": "user", "content": prompt}],
                                             max_tokens = 2,
                                             n=2)
                print('Sleep 1 second')
                time.sleep(2)
                print('Sleep done')
            except:
                print('Sleep 10 seconds')
                time.sleep(10)
                print('Finished sleep')
                responses = openai.ChatCompletion.create(model="gpt-3.5-turbo",
                                             messages=[{"role": "user", "content": prompt}],
                                             max_tokens = 2,
                                             n=2)
                print('Got through in the end')
                pass
            all_responses.extend([i['message']['content'] for i in responses['choices']])
        mega_responses.append(all_responses)
        save = pd.DataFrame(mega_responses, 
                                columns= ['id','text', 'class', 
                                        'chatGPT_few_default_1', 'chatGPT_few_default_2',
                                        'chatGPT_zero_default_1','chatGPT_zero_default_2',
                                        'chatGPT_zero_specific_1','chatGPT_zero_specific_2',
                                        'chatGPT_few_specific_1','chatGPT_few_specific_2',
                                        'chatGPT_counterfactual_default_1', 'chatGPT_counterfactual_default_2',
                                        'chatGPT_counterfactual_specific_1', 'chatGPT_counterfactual_specific_2'])
        save.to_csv(path + df.attrs['name'] + '_wip5.csv', index = False)
        print(f'{index + 1}/{len(df)} completed')
    labeled_df = pd.DataFrame(mega_responses, 
                              columns = ['id','text', 'class', 
                                        'chatGPT_few_default_1', 'chatGPT_few_default_2', 
                                        'chatGPT_zero_default_1','chatGPT_zero_default_2', 
                                        'chatGPT_zero_specific_1','chatGPT_zero_specific_2',
                                        'chatGPT_few_specific_1','chatGPT_few_specific_2',
                                        'chatGPT_counterfactual_default_1', 'chatGPT_counterfactual_default_2',
                                        'chatGPT_counterfactual_specific_1', 'chatGPT_counterfactual_specific_2'])
    labeled_df.to_csv(path + df.attrs['name'] + '_labeled_chatgpt.csv', index = False)

Sleep 1 second
Sleep done
Sleep 1 second
Sleep done
Sleep 1 second
Sleep done
Sleep 1 second
Sleep done
Sleep 1 second
Sleep done
Sleep 1 second
Sleep done
1/3 completed
Sleep 1 second
Sleep done
Sleep 1 second
Sleep done
Sleep 1 second
Sleep done
Sleep 1 second
Sleep done
Sleep 1 second
Sleep done
Sleep 1 second
Sleep done
2/3 completed
Sleep 1 second
Sleep done
Sleep 1 second
Sleep done
Sleep 1 second
Sleep done
Sleep 1 second
Sleep done
Sleep 1 second
Sleep done
Sleep 1 second
Sleep done
3/3 completed


In [123]:
import pandas as pd
import numpy as np
path = "/home/ubuntu/Konstanz/SILLM/Data/"
reddit_full_labeled = pd.read_csv(path + 'reddit_wip.csv').append(pd.read_csv(path + 'reddit_wip1.csv')).append(pd.read_csv(path + 'reddit_wip2.csv')).append(pd.read_csv(path + 'reddit_wip3.csv')).append(pd.read_csv(path + 'reddit_wip4.csv')).append(pd.read_csv(path + 'reddit_wip5.csv'))
reddit_full_labeled['class'] = reddit_full_labeled['class'].notna().astype(int)
gab_full_labeled = pd.concat([pd.read_csv(path + 'gab_wip.csv'), pd.read_csv(path + 'gab_wip1.csv'), pd.read_csv(path + 'gab_wip2.csv'), pd.read_csv(path + 'gab_wip3.csv')])
gab_full_labeled['class'] = gab_full_labeled['class'].notna().astype(int)
hasoc_full_labeled = pd.concat([pd.read_csv(path + 'hasoc_wip.csv'), pd.read_csv(path + 'hasoc_wip1.csv'), pd.read_csv(path + 'hasoc_wip2.csv'), pd.read_csv(path + 'hasoc_wip3.csv'), pd.read_csv(path + 'hasoc_wip4.csv')])
hasoc_full_labeled['class'] = hasoc_full_labeled['class'].map({'not hate':0, 'hate':1})
hateval_full_labeled = pd.concat([pd.read_csv(path + 'hateval_wip.csv'), pd.read_csv(path + 'hateval_wip1.csv'), pd.read_csv(path + 'hateval_wip2.csv'), pd.read_csv(path + 'hateval_wip3.csv'), pd.read_csv(path + 'hateval_wip4.csv')])
pd.unique(hateval_full_labeled[['chatGPT_few_default_1', 'chatGPT_few_default_2',
       'chatGPT_zero_default_1', 'chatGPT_zero_default_2',
       'chatGPT_zero_specific_1', 'chatGPT_zero_specific_2',
       'chatGPT_few_specific_1', 'chatGPT_few_specific_2',
       'chatGPT_counterfactual_default_1', 'chatGPT_counterfactual_default_2',
       'chatGPT_counterfactual_specific_1',
       'chatGPT_counterfactual_specific_2']].values.ravel('K'))



reddit_flan = pd.read_csv(path + 'reddit_labeled_flan.csv', names = ['id','text', 'class', 
                                        'flan_xl_few_default_1', 'flan_xl_few_default_2','flan_xl_few_default_3', 
                                        'flan_xl_zero_default_1','flan_xl_zero_default_2', 'flan_xl_zero_default_3',
                                        'flan_xl_zero_specific_1','flan_xl_zero_specific_2','flan_xl_zero_specific_3', 
                                        'flan_xl_few_specific_1','flan_xl_few_specific_2','flan_xl_few_specific_3', 
                                        'flan_xl_counterfactual_default_1', 'flan_xl_counterfactual_default_2','flan_xl_counterfactual_default_3',
                                        'flan_xl_counterfactual_specific_1', 'flan_xl_counterfactual_specific_2','flan_xl_counterfactual_specific_3']).iloc[:100]
reddit_full_labeled = reddit_full_labeled.merge(reddit_flan, on = 'text').rename(columns = {'id_x':'id', 'class_x':'class'})
gab_flan = pd.read_csv(path + 'gab_labeled_flan.csv', header = 0, names = ['id','text', 'class', 
                                        'flan_xl_few_default_1', 'flan_xl_few_default_2','flan_xl_few_default_3', 
                                        'flan_xl_zero_default_1','flan_xl_zero_default_2', 'flan_xl_zero_default_3',
                                        'flan_xl_zero_specific_1','flan_xl_zero_specific_2','flan_xl_zero_specific_3', 
                                        'flan_xl_few_specific_1','flan_xl_few_specific_2','flan_xl_few_specific_3', 
                                        'flan_xl_counterfactual_default_1', 'flan_xl_counterfactual_default_2','flan_xl_counterfactual_default_3',
                                        'flan_xl_counterfactual_specific_1', 'flan_xl_counterfactual_specific_2','flan_xl_counterfactual_specific_3']).iloc[:100]
# gab_full_labeled = reddit_full_labeled.merge(gab_flan, on = 'text').rename(columns = {'id_x':'id', 'class_x':'class'})
hasoc_flan = pd.read_csv(path + 'hasoc_labeled_flan.csv', names = ['id','text', 'class', 
                                        'flan_xl_few_default_1', 'flan_xl_few_default_2','flan_xl_few_default_3', 
                                        'flan_xl_zero_default_1','flan_xl_zero_default_2', 'flan_xl_zero_default_3',
                                        'flan_xl_zero_specific_1','flan_xl_zero_specific_2','flan_xl_zero_specific_3', 
                                        'flan_xl_few_specific_1','flan_xl_few_specific_2','flan_xl_few_specific_3', 
                                        'flan_xl_counterfactual_default_1', 'flan_xl_counterfactual_default_2','flan_xl_counterfactual_default_3',
                                        'flan_xl_counterfactual_specific_1', 'flan_xl_counterfactual_specific_2','flan_xl_counterfactual_specific_3']).iloc[:100]
hasoc_full_labeled = hasoc_full_labeled.merge(hasoc_flan, on = 'text').rename(columns = {'id_x':'id', 'class_x':'class'})
hateval_flan = pd.read_csv(path + 'hateval_labeled_flan.csv', names = ['id','text', 'class', 
                                        'flan_xl_few_default_1', 'flan_xl_few_default_2','flan_xl_few_default_3', 
                                        'flan_xl_zero_default_1','flan_xl_zero_default_2', 'flan_xl_zero_default_3',
                                        'flan_xl_zero_specific_1','flan_xl_zero_specific_2','flan_xl_zero_specific_3', 
                                        'flan_xl_few_specific_1','flan_xl_few_specific_2','flan_xl_few_specific_3', 
                                        'flan_xl_counterfactual_default_1', 'flan_xl_counterfactual_default_2','flan_xl_counterfactual_default_3',
                                        'flan_xl_counterfactual_specific_1', 'flan_xl_counterfactual_specific_2','flan_xl_counterfactual_specific_3']).iloc[:100]
hateval_full_labeled = hateval_full_labeled.merge(hateval_flan, on = 'text').rename(columns = {'id_x':'id', 'class_x':'class'})

# Define the custom function for transformation
def transform_value(value):
    if isinstance(value, str):
        value_lower = value.lower()
        if 'not hate' in value_lower or '2' in value_lower:
            return 0
        elif 'hate' in value_lower or '1' in value_lower:
            return 1
    return 3

# Apply the transformation to all dataframes
dfs = [reddit_full_labeled, gab_full_labeled, hasoc_full_labeled, hateval_full_labeled]
columns = ['chatGPT_few_default_1', 'chatGPT_few_default_2',
       'chatGPT_zero_default_1', 'chatGPT_zero_default_2',
       'chatGPT_zero_specific_1', 'chatGPT_zero_specific_2',
       'chatGPT_few_specific_1', 'chatGPT_few_specific_2',
       'chatGPT_counterfactual_default_1', 'chatGPT_counterfactual_default_2',
       'chatGPT_counterfactual_specific_1',
       'chatGPT_counterfactual_specific_2']

for df in dfs:
    df[columns] = df[columns].applymap(transform_value)




dfs = [reddit_full_labeled, gab_flan, hasoc_full_labeled, hateval_full_labeled]
columns = ['flan_xl_few_default_1', 'flan_xl_few_default_2', 'flan_xl_few_default_3', 'flan_xl_zero_default_1', 'flan_xl_zero_default_2', 'flan_xl_zero_default_3', 'flan_xl_zero_specific_1', 'flan_xl_zero_specific_2', 'flan_xl_zero_specific_3', 'flan_xl_few_specific_1', 'flan_xl_few_specific_2', 'flan_xl_few_specific_3', 'flan_xl_counterfactual_default_1', 'flan_xl_counterfactual_default_2','flan_xl_counterfactual_default_3', 'flan_xl_counterfactual_specific_1','flan_xl_counterfactual_specific_2','flan_xl_counterfactual_specific_3']

for df in dfs: 
    for column in columns:
        df[column] = df[column].map({'not hate':0, 'hate':1})
    df.fillna(3, inplace=True)

gab_flan

/tmp/ipykernel_2059162/2868932758.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  reddit_full_labeled = pd.read_csv(path + 'reddit_wip.csv').append(pd.read_csv(path + 'reddit_wip1.csv')).append(pd.read_csv(path + 'reddit_wip2.csv')).append(pd.read_csv(path + 'reddit_wip3.csv')).append(pd.read_csv(path + 'reddit_wip4.csv')).append(pd.read_csv(path + 'reddit_wip5.csv'))
/tmp/ipykernel_2059162/2868932758.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  reddit_full_labeled = pd.read_csv(path + 'reddit_wip.csv').append(pd.read_csv(path + 'reddit_wip1.csv')).append(pd.read_csv(path + 'reddit_wip2.csv')).append(pd.read_csv(path + 'reddit_wip3.csv')).append(pd.read_csv(path + 'reddit_wip4.csv')).append(pd.read_csv(path + 'reddit_wip5.csv'))
/tmp/ipykernel_2059162/2868932758.py:4: FutureWarning: The frame.append 

,id,text,class,flan_xl_few_default_1,flan_xl_few_default_2,flan_xl_few_default_3,flan_xl_zero_default_1,flan_xl_zero_default_2,flan_xl_zero_default_3,flan_xl_zero_specific_1,...,flan_xl_zero_specific_3,flan_xl_few_specific_1,flan_xl_few_specific_2,flan_xl_few_specific_3,flan_xl_counterfactual_default_1,flan_xl_counterfactual_default_2,flan_xl_counterfactual_default_3,flan_xl_counterfactual_specific_1,flan_xl_counterfactual_specific_2,flan_xl_counterfactual_specific_3
0,4252 1. 38332766\n2. \t38342937\n3. \t\t38...,1. If Democrats get control Maxine Waters will...,[3],1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,4252 1. 38332766\n2. \t38342937\n3. \t\t38...,"1. Yeah really, this isnt the first time. #CH ...",[1],1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,4252 1. 38332766\n2. \t38342937\n3. \t\t38...,1. Behold! I have discovered @Microchip's true...,[2],0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4252 1. 38332766\n2. \t38342937\n3. \t\t38...,1. FUCK U nIGGERS! 🖕🐒 https://www.youtube.co...,"[1, 2]",1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,4252 1. 38332766\n2. \t38342937\n3. \t\t38...,"1. Well, nobody ever said it took that many br...",[2],0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,4252 1. 38332766\n2. \t38342937\n3. \t\t38...,1. WTH? EVIL Idiot Mazie Hirono Says Kavanaugh...,[2],1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
96,4252 1. 38332766\n2. \t38342937\n3. \t\t38...,1. In anticipation of #BohemianRhapsody releas...,[3],0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
97,4252 1. 38332766\n2. \t38342937\n3. \t\t38...,"1. right, man i was on the streets at 13 handi...",[1],1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
98,4252 1. 38332766\n2. \t38342937\n3. \t\t38...,1. GM All Gabbers Have a safe & blessed Wednes...,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [124]:
from sklearn.metrics import f1_score

f1_scores = []

columns = ['chatGPT_few_default_1', 'chatGPT_few_default_2',
       'chatGPT_zero_default_1', 'chatGPT_zero_default_2',
       'chatGPT_zero_specific_1', 'chatGPT_zero_specific_2',
       'chatGPT_few_specific_1', 'chatGPT_few_specific_2',
       'chatGPT_counterfactual_default_1', 'chatGPT_counterfactual_default_2',
       'chatGPT_counterfactual_specific_1',
       'chatGPT_counterfactual_specific_2', 'flan_xl_few_default_1', 'flan_xl_few_default_2', 'flan_xl_few_default_3', 'flan_xl_zero_default_1', 'flan_xl_zero_default_2', 'flan_xl_zero_default_3', 'flan_xl_zero_specific_1', 'flan_xl_zero_specific_2', 'flan_xl_zero_specific_3', 'flan_xl_few_specific_1', 'flan_xl_few_specific_2', 'flan_xl_few_specific_3', 'flan_xl_counterfactual_default_1', 'flan_xl_counterfactual_default_2','flan_xl_counterfactual_default_3', 'flan_xl_counterfactual_specific_1','flan_xl_counterfactual_specific_2','flan_xl_counterfactual_specific_3']
dfs = [reddit_full_labeled, hasoc_full_labeled, hateval_full_labeled]
for df in dfs:
    results = []
    for model in columns:
        print(model)
        f1 = f1_score(df['class'], df[model], average='macro')
        print(f1)
        results.append(round(f1, 3)) 
    f1_scores.append(results)



# flan_columns = ['flan_xl_few_default_1', 'flan_xl_few_default_2', 'flan_xl_few_default_3', 'flan_xl_zero_default_1', 'flan_xl_zero_default_2', 'flan_xl_zero_default_3', 'flan_xl_zero_specific_1', 'flan_xl_zero_specific_2', 'flan_xl_zero_specific_3', 'flan_xl_few_specific_1', 'flan_xl_few_specific_2', 'flan_xl_few_specific_3', 'flan_xl_counterfactual_default_1', 'flan_xl_counterfactual_default_2','flan_xl_counterfactual_default_3', 'flan_xl_counterfactual_specific_1','flan_xl_counterfactual_specific_2','flan_xl_counterfactual_specific_3']
# gpt_columns = ['chatGPT_few_default_1', 'chatGPT_few_default_2',
    #    'chatGPT_zero_default_1', 'chatGPT_zero_default_2',
    #    'chatGPT_zero_specific_1', 'chatGPT_zero_specific_2',
    #    'chatGPT_few_specific_1', 'chatGPT_few_specific_2',
    #    'chatGPT_counterfactual_default_1', 'chatGPT_counterfactual_default_2',
    #    'chatGPT_counterfactual_specific_1',
    #    'chatGPT_counterfactual_specific_2']

# results_gab = []
# for model in gpt_columns:
#     f1 = f1_score(gab_full_labeled['class'], df[model], average='macro')
#     results_gab.append(round(f1, 3))
# for model in flan_columns:
#     f1 = f1_score(gab_flan['class'], df[model], average='macro')
#     results_gab.append(round(f1, 3))




f1_scores_df = pd.DataFrame(f1_scores, columns = columns, index = ['reddit', 'hasoc', 'hateval'])
f1_scores_df


chatGPT_few_default_1
0.3808353808353808
chatGPT_few_default_2
0.32414369256474523
chatGPT_zero_default_1
0.36759259259259264
chatGPT_zero_default_2
0.3661616161616162
chatGPT_zero_specific_1
0.5613924050632911
chatGPT_zero_specific_2
0.36679174484052535
chatGPT_few_specific_1
0.5689404934687954
chatGPT_few_specific_2
0.4028085735402808
chatGPT_counterfactual_default_1
0.5008403361344538
chatGPT_counterfactual_default_2
0.35239865097391426
chatGPT_counterfactual_specific_1
0.47816265060240964
chatGPT_counterfactual_specific_2
0.3345436940208182
flan_xl_few_default_1
0.41408163265306125
flan_xl_few_default_2
0.41408163265306125
flan_xl_few_default_3
0.41408163265306125
flan_xl_zero_default_1
0.31781149037786205
flan_xl_zero_default_2
0.31781149037786205
flan_xl_zero_default_3
0.31781149037786205
flan_xl_zero_specific_1
0.47037037037037044
flan_xl_zero_specific_2
0.47037037037037044
flan_xl_zero_specific_3
0.47037037037037044
flan_xl_few_specific_1
0.4233009708737864
flan_xl_few_specific

,chatGPT_few_default_1,chatGPT_few_default_2,chatGPT_zero_default_1,chatGPT_zero_default_2,chatGPT_zero_specific_1,chatGPT_zero_specific_2,chatGPT_few_specific_1,chatGPT_few_specific_2,chatGPT_counterfactual_default_1,chatGPT_counterfactual_default_2,...,flan_xl_zero_specific_3,flan_xl_few_specific_1,flan_xl_few_specific_2,flan_xl_few_specific_3,flan_xl_counterfactual_default_1,flan_xl_counterfactual_default_2,flan_xl_counterfactual_default_3,flan_xl_counterfactual_specific_1,flan_xl_counterfactual_specific_2,flan_xl_counterfactual_specific_3
reddit,0.381,0.324,0.368,0.366,0.561,0.367,0.569,0.403,0.501,0.352,...,0.470,0.423,0.423,0.423,0.526,0.526,0.526,0.500,0.500,0.500
hasoc,0.306,0.261,0.337,0.288,0.292,0.312,0.272,0.276,0.196,0.205,...,0.441,0.446,0.446,0.446,0.426,0.426,0.426,0.431,0.431,0.431
hateval,0.435,0.440,0.437,0.456,0.456,0.456,0.439,0.462,0.460,0.444,...,0.700,0.726,0.726,0.726,0.677,0.677,0.677,0.677,0.677,0.677


**Do your results for mode A have the same performance as those reported in the paper (figure 1)? [2 points]**


**Is one of the LLMs better than the other one? [2 points]**
Flan is much more consistent across all of the runs, which is a very good property to have. I would rather have a more consistent if not as accurate model versus a potetially better model that has a larger varience in its answers. The values are also higher or at least very similar between Flan and chatGPT.


**Does varying the definition improve performance? [2 points]**
Yes, the few shot models responded very positively to a more precise definition of the dataset. This was a definition I came up with through reading the papers and also adding some of my own research into the platforms where the data came from. The counterfactual models don't really have much of a difference between the default and the specific definitions as the counterfactuals 

**Which prompting technique — zero-shot, few-shot — works best? [2 points] Bonus: also compare against counterfactual prompting [Bonus 1 point]**
Few shot consistenly outperforms zero shot. This makes sense as we are basically expanding upon the definition of the task we want the LLMs to perform. Counterfactual did better than both of these however, as we show what specific words or phrases are considered hate speech. For example, I mostly added/subtracted slurs and rude language, which is a prominent feature of hate speech. 
